In [2]:
import pandas as pd
import warnings; warnings.simplefilter('ignore')

# PreProcessing Functions

In [3]:
def extract_traffic_source_columns(df):
    df['isTrueDirect']=df['isTrueDirect'].fillna(False)
    df['isTrueDirect']=df['isTrueDirect'].astype(int)
    
    is_source_mall_googleplex = (df['source']=='mall.googleplex.com').astype(int)
    is_source_mall_googleplex.name = 'is_source_googleplex'    
    
    is_source_direct = (df['source']=='(direct)').astype(int)
    is_source_direct.name = 'is_source_direct'
    
    is_source_google = (df['source']=='(direct)').astype(int)
    is_source_google.name = 'is_source_google'
    
    is_medium_referral = (df['medium']=='referral').astype(int)
    is_medium_referral.name = 'is_medium_referral'
    is_medium_none = (df['medium']=='(none)').astype(int)
    is_medium_none.name = 'is_medium_none'
    is_medium_organic = (df['medium']=='organic').astype(int)
    is_medium_organic.name="is_medium_organic"
    
    df = df.drop(columns = ['adContent','adwordsClickInfo','campaign', 'keyword', 'medium', 'referralPath',
                      'source'])
    
    return pd.concat([df,is_source_mall_googleplex,is_source_direct,is_source_google,is_medium_referral,
                    is_medium_none,is_medium_organic],axis=1)

In [4]:
def get_region_columns(df):
    df['is_bayarea'] = (df['metro'] =='San Francisco-Oakland-San Jose CA').astype(int)
    dummies = pd.get_dummies(df['continent'])
    dummies = dummies.drop(['(not set)'],axis=1)
    df = df.drop(['continent','city','subContinent','region','country','metro','networkDomain'],axis=1) 
    new_df = pd.concat([df,dummies],axis=1)
    return new_df

In [5]:
def extract_activities(df):
    df['is_transaction'] = 1 - df['transactionRevenue'].isna()
    df[['bounces', 'hits', 'pageviews', 'visitNumber']] = df[['bounces', 'hits', 'pageviews', 'visitNumber']].fillna(0)
    return df
    

In [6]:
def time_value(df):
    import datetime as dt
    time = pd.to_datetime(df.loc[:,'visitStartTime'], unit='s')
    year = time.dt.year
    year = pd.get_dummies(year)
    month = time.dt.month
    q1 = (month <= 3) & (month >= 1)
    q2 = (month <= 6) & (month >= 4)
    q3 = (month <= 9) & (month >= 7)
    q4 = (month <= 12) & (month >= 10)
    dow = time.dt.weekday
    weekday = (dow <= 5) & (dow >=1)
    hour = time.dt.hour
    h_0_6 = (hour <= 6) & (hour >= 0)
    h_7_12 = (hour <= 12) & (hour >= 7)
    h_13_17 = (hour <= 17) & (hour >= 13)
    h_18_23 = (hour <= 23) & (hour >= 18)
    new_df = pd.DataFrame()
    new_df['q1'] = q1.astype(int)
    new_df['q2'] = q2.astype(int)
    new_df['q3'] = q3.astype(int)
    new_df['q4'] = q4.astype(int)
    new_df['weekday'] = weekday.astype(int)
    new_df['h_0_6'] = h_0_6.astype(int)
    new_df['h_7_12'] = h_7_12.astype(int)
    new_df['h_13_17'] = h_13_17.astype(int)
    new_df['h_18_23'] = h_18_23.astype(int)
    return new_df

def device(df):
    browser_chrome = (df['browser']=='Chrome').astype(int)
    is_mobile = df['isMobile'].astype(int)
    system_google = ((df['operatingSystem']=='Android')|(df['operatingSystem']=='Chrome OS')).astype(int)
    new_df = pd.DataFrame()
    new_df['browser_chrome'] = browser_chrome
    new_df['is_mobile'] = is_mobile
    new_df['system_google'] = system_google
    return new_df

In [7]:
def normalization(col):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    col = np.array(col.fillna(0)).reshape(-1,1)
    col = scaler.fit_transform(col)
    return col

In [8]:
def preprocessing(inputfile):
    df = pd.read_csv(inputfile)
    
    df.sort_values(['fullVisitorId', 'visitStartTime'], ascending=True, inplace=True)
    df['last_session_1'] = (
    df['visitStartTime'] - df[['fullVisitorId', 'visitStartTime']].groupby('fullVisitorId')['visitStartTime'].shift(1)
)
    df['next_session_1']=(
    -1*(df['visitStartTime'] - df[['fullVisitorId', 'visitStartTime']].groupby('fullVisitorId')['visitStartTime'].shift(-1))
)

    #df[['next_session_1','last_session_1']] = df[['next_session_1','last_session_1']]/86400
    df[['next_session_1','last_session_1']] = df[['next_session_1','last_session_1']].fillna(0)
    print(df.columns)
    df = extract_traffic_source_columns(df)
    # print("Congrats Brian\n"+df.columns)
    df = get_region_columns(df)
    # print("Congrats Ziyu\n"+df.columns)
    df = extract_activities(df)
    # print("Congrats Yixin\n"+df.columns)
    col_time = time_value(df)
    col_device = device(df)
    
    df_base_model = df[['bounces', 'hits', 'pageviews', 'visitNumber',
                       'is_source_googleplex','is_source_direct','is_source_google','is_medium_referral',
                    'is_medium_none','is_medium_organic',
                        'is_bayarea','Africa','Americas','Asia','Europe','Oceania',
                        'next_session_1','last_session_1'
                       ]]
    df_base_model = pd.concat([df_base_model,col_time],axis=1)
    df_base_model = pd.concat([df_base_model,col_device],axis=1)
    df_istransaction = df['is_transaction']
    df_transaction = normalization(df['transactionRevenue'])
    # print("Congrats Xi\n"+df_base_model.columns)
    return df_base_model, df_istransaction, df_transaction

# Create DataFrame

In [ ]:
inputfile = '../data/train_flatten_with_ts.csv'
df_base_model, df_istransaction, df_transaction = preprocessing(inputfile)

# Model Fitting

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import precision_recall_fscore_support

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    df_base_model, df_istransaction, test_size=0.33, random_state=42)
estimator = EllipticEnvelope()
y_predicted = estimator.fit_predict(X_train, y_train)
precision_recall_fscore_support(y_train, y_predicted, average='macro')

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:646: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-214.687840879731880 > -258.877618087802489). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-192.250928241014947 > -208.871394867089293). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-p

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-189.057117641256212 > -192.689845954215457). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-225.108801852780573 > -228.110577894071042). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-203.011919237679393 > -208.268492742666069). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-162.989119582789669 > -189.149449613069777). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-176.430300752094155 > -195.378929797277920). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-167.060925111865004 > -170.942474582649936). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-184.881720581951129 > -185.071888760315005). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-188.047269180957500 > -188.842306557487518). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-190.106008712917486 > -199.089629111049476). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-184.474187427808715 > -221.229961205467021). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-179.245021649481998 > -186.004943548439229). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-199.204358763786360 > -218.435103774962016). You may want to try with a higher value of support_frac

...

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-169.649679434177273 > -199.602250905341265). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-186.592762367483800 > -206.445541425700100). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-171.306694761432965 > -199.061336299503211). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-224.612731455378650 > -238.958290180309177). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-188.601364587643729 > -197.699731707544288). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-191.004931796512523 > -194.713493966527977). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-179.435794686151439 > -193.964829953357537). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-226.157815488931789 > -261.199637009463913). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-183.948703575240643 > -190.849030748844484). You may want to try with a higher value of support_frac

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-179.713644134158386 > -204.667719002775556). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-188.649074080015964 > -191.531288318253615). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-170.026322250240611 > -177.360419808106286). You may want to try with a higher value of support_frac

(0.0033400501374559097, 0.23875114784205695, 0.006587937197411881, None)

# Regression Model Fit

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_base_model, df_transaction, test_size=0.33, random_state=42)
glm = LinearRegression()
reg = glm.fit(X_train, y_train)
reg.score(X_train, y_train)